In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt

import pandas as pd
import os

from helper_funcs import *

In [ ]:
f = rfftfreq(32768, 1/44100)[0:8192]
noise_floors = []
species_list = []
filepaths = []

rows = []

for noise_floor, species, filepath in zip(noise_floors, species_list, filepaths):
    d = gen_spectrum(f=f, species=species, noise_floor=noise_floor, noise_sigma=0.01, f0_dist='chi', plot=True)
    
    
    # rows.append(d)
    
# synth_transfer_df = pd.DataFrame(rows)
# synth_transfer_df.to_parquet("synth_transfer_df.parquet", engine='pyarrow')

In [ ]:

transfer_samples = gen_samples(df=synth_transfer_df, hwhm_max=150, snr_max=20)
general_samples = gen_samples(df=synth_general_df, hwhm_max=150, snr_max=20)
samples = np.concatenate((transfer_samples, general_samples), axis=0)

In [ ]:
# VISUALIZE CHI SQUARE POSITION DIST
# Parameters for the chi-square distribution
f0_chi_dof = 10  # k (degrees of freedom)
num_samples = 10000     # Number of samples to generate

# Generate samples from a chi-square distribution
f0_chi_og_pivot = 20
f0_chi_new_pivot = 6000
chi_square_samples = np.random.chisquare(df=f0_chi_dof, size=num_samples)*f0_chi_new_pivot/f0_chi_og_pivot

# Plot the histogram of the samples
plt.hist(chi_square_samples, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
plt.title(f'Chi-Square Distribution (k={f0_chi_dof})')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
# VISUALIZE LORENTZIAN
f = rfftfreq(32768, 1/44100)
f0 = f[5000]
snr = 20
hwhm = 1

plt.plot(f, lorentzian(f, f0, hwhm, snr))
plt.plot(f, np.ones(len(f)) * 10)
plt.xlim(f0-2, f0+2)
plt.show()

In [ ]:
# VISUALIZE NOISE FLOOR
# Get frequency axis
f = rfftfreq(32768, 1/44100)[0:8192]

# First navigate to our directory
directory_path = os.path.join("Data", "processed_df.parquet")
# Load the dataframe
df = pd.read_parquet(directory_path)

df = df[df['filepath'].str.contains("UWO Data")]
df = df[df['filepath'].str.contains("waveform")]
df = df.sample(frac=1).reset_index(drop=True)
spectrums = df['spectrum'].to_list()
filepaths = df['filepath'].to_list()
species = df['species'].to_list()

In [ ]:
frac_broad= 0.3
smoothing_sigma = 150
fit_close_cutoff = 300

i = 0

for spectrum, filepath in zip(spectrums, filepaths):
    i+=1
    if i > 10:
        break
    
    
    noise_floor = get_noise_floor(f, spectrum)
    plt.plot(f, spectrum, label='Original Spectrum')
    # plt.plot(f, min_spectrum, 'black', label='min_spectrum')
    # plt.plot(f, fit_broad,'g--',label='Broad LOWESS Fit')
    # plt.plot(f, fit_close,'r--',label='Close LOWESS Fit')
    plt.plot(f, noise_floor,'black',label='Noise Floor')
    plt.legend()
    plt.title(filepath)
    plt.savefig(filepath + ".png")
    plt.show()
    